# **Distribución % de créditos por organismo**

In [1]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()

# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    # Caso de cuando no son las alfa q
    if nombre.startswith("df_") and "financiamientos" in nombre and "vallarta" in nombre
]

# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_financiamientos_2019_vallarta',
 'df_financiamientos_2020_vallarta',
 'df_financiamientos_2021_vallarta',
 'df_financiamientos_2022_vallarta',
 'df_financiamientos_2023_vallarta',
 'df_financiamientos_2024_vallarta']

## 2022

In [2]:
finance = df_financiamientos_2022_vallarta[['id','organismo']]
finance

,id,organismo
0,1,1
1,2,1
2,3,1
3,4,1
4,5,1
...,...,...
1269,1270,3
1270,1271,3
1271,1272,3
1272,1273,3


In [3]:
# Convertir la columna 'organismo' a cadena de texto (str)
#finance['organismo'] = finance['organismo'].astype(str)
finance.loc[:, 'organismo'] = finance['organismo'].astype(str)

# Crear un diccionario con las correspondencias entre número y categoría
categorias = {
    '1': 'INFONAVIT',
    '2': 'CNBV',
    '3': 'FOVISSTE',
    '5': 'CONAVI',
    '7': 'BANJERCITO',
    '26': 'INSUS'
}
# Reemplazar los valores de 'organismo' con el texto correspondiente utilizando replace
#finance['organismo'] = finance['organismo'].replace(categorias)
finance.loc[:, 'organismo'] = finance['organismo'].replace(categorias)

# Reemplazar cualquier otro número que no esté en el diccionario con '-'
#finance['organismo'] = finance['organismo'].replace(to_replace=r'^\d+$', value='-', regex=True)
finance.loc[:, 'organismo'] = finance['organismo'].replace(to_replace=r'^\d+$', value='-', regex=True)

# Eliminar la fila donde rango_ingresos es '0'
finance = finance[finance['organismo'] != '-']

# Obtener el conteo de registros por cada categoría en la columna 'organismo'
conteo_organismos = finance['organismo'].value_counts().reset_index()
conteo_organismos.columns = ['organismo', 'conteo']


# Asegurarse de que todas las categorías estén presentes en el conteo, incluso si tienen un conteo de 0
for categoria in categorias.values():
    if categoria not in conteo_organismos['organismo'].unique():
        nuevo_registro = pd.DataFrame({'organismo': [categoria], 'conteo': [0]})
        conteo_organismos = pd.concat([conteo_organismos, nuevo_registro], ignore_index=True)

# Ordenar el DataFrame por las categorías en orden ascendente
conteo_organismos = conteo_organismos.sort_values(by='organismo').reset_index(drop=True)

print(conteo_organismos)

    organismo  conteo
0  BANJERCITO       0
1        CNBV     686
2      CONAVI       0
3    FOVISSTE      57
4   INFONAVIT     518
5       INSUS       0


C:\Users\yoe11\AppData\Local\Temp\ipykernel_13872\2664051192.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['1' '1' '1' ... '3' '3' '3']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  finance.loc[:, 'organismo'] = finance['organismo'].astype(str)


In [4]:
# Crear un DataFrame con el conteo de registros por cada organismo
df_conteo_organismos = finance['organismo'].value_counts().reset_index()
df_conteo_organismos.columns = ['organismo', 'conteo']

# Calcular el total de registros en el DataFrame original
total_registros = len(finance)

# Calcular el promedio que ocupa cada organismo con respecto al total de registros
df_conteo_organismos['promedio'] = round((df_conteo_organismos['conteo'] * 100) / total_registros,2)
df_conteo_organismos

,organismo,conteo,promedio
0,CNBV,686,54.40
1,INFONAVIT,518,41.08
2,FOVISSTE,57,4.52


In [5]:
# Calcular el total de registros en el DataFrame original
total_registros = len(finance)

# Calcular el promedio que ocupa cada organismo con respecto al total de registros
conteo_organismos['promedio'] = round((conteo_organismos['conteo'] * 100) / total_registros,2)
conteo_organismos

,organismo,conteo,promedio
0,BANJERCITO,0,0.00
1,CNBV,686,54.40
2,CONAVI,0,0.00
3,FOVISSTE,57,4.52
4,INFONAVIT,518,41.08
5,INSUS,0,0.00


In [6]:
f_2022 = finance.size
cont = df_conteo_organismos['conteo'].sum()
print("Hay ", f_2022, " registros en la base")
print("Hay ", cont, " registros al contar")

Hay  2522  registros en la base
Hay  1261  registros al contar


In [7]:
# Definir el diccionario de orden de organismos
orden_organismos = {
    'INFONAVIT': 1,
    'CNBV': 2,
    'FOVISSTE': 3,
    'BANJERCITO': 4
}

#df_conteo_organismos['orden'] = df_conteo_organismos['organismo'].map(orden_organismos)
df_conteo_organismos.loc[:, 'orden'] = df_conteo_organismos['organismo'].map(orden_organismos)

# Ordenar el DataFrame por el orden de los organismos
df_conteo_organismos = df_conteo_organismos.sort_values(by='orden')
df_conteo_organismos = df_conteo_organismos.drop(columns=['orden'])
df_conteo_organismos

,organismo,conteo,promedio
1,INFONAVIT,518,41.08
0,CNBV,686,54.40
2,FOVISSTE,57,4.52


In [8]:

labels = df_conteo_organismos['organismo']
values = df_conteo_organismos['promedio']
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']

fig = go.Figure()
fig.add_trace(go.Bar(
        x=labels, 
        y=values, 
        marker_color=colores,
        text=values,  # Texto que se mostrará en las barras (porcentaje)
        textposition='outside',  # Posición del texto (puede ser 'inside' o 'outside')
        #texttemplate='%{text:.2f}%', 
    )
)    
fig.update_layout(
    #barmode='stack', 
    #title='Porcentaje de créditos por organismo (2022)',
    #title_font_size = 22,
    yaxis=dict(
        title='%', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y         
    ),  margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    legend=dict(
        yanchor="top",
        y=0.99,
    )
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets\graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    # Configuración personalizada para la gráfica
    config = {
        'displaylogo': False,  # Ocultar el logo de Plotly
        'modeBarButtonsToRemove': [
            'toImage',       # Botón para guardar como imagen
            'select2d',      # Box select
            'lasso2d',       # Lasso select
            'resetScale2d',  # Reset Axes
        ]
    }

    # Guardar la gráfica como archivo HTML
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html', config=config)
guardar_grafico_como_html(fig, 'g_bar_creditos_organismo_2022', carpeta='assets\graficas')
fig.show()

<>:30: SyntaxWarning: invalid escape sequence '\g'
<>:47: SyntaxWarning: invalid escape sequence '\g'
<>:30: SyntaxWarning: invalid escape sequence '\g'
<>:47: SyntaxWarning: invalid escape sequence '\g'
C:\Users\yoe11\AppData\Local\Temp\ipykernel_13872\2286633222.py:30: SyntaxWarning: invalid escape sequence '\g'
  def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets\graficas'):
C:\Users\yoe11\AppData\Local\Temp\ipykernel_13872\2286633222.py:47: SyntaxWarning: invalid escape sequence '\g'
  guardar_grafico_como_html(fig, 'g_bar_creditos_organismo_2022', carpeta='assets\graficas')


## 2023

In [9]:
finance = df_financiamientos_2023_vallarta[['id','organismo']]
finance

,id,organismo
0,1,1
1,2,1
2,3,1
3,4,1
4,5,1
...,...,...
1379,1380,2
1380,1381,3
1381,1382,3
1382,1383,4


In [10]:
# Convertir la columna 'organismo' a cadena de texto (str)
#finance['organismo'] = finance['organismo'].astype(str)
finance.loc[:, 'organismo'] = finance['organismo'].astype(str)

# Crear un diccionario con las correspondencias entre número y categoría
categorias = {
    '1': 'INFONAVIT',
    '2': 'CNBV',
    '3': 'FOVISSTE',
    '5': 'CONAVI',
    '7': 'BANJERCITO',
    '26': 'INSUS'
}
# Reemplazar los valores de 'organismo' con el texto correspondiente utilizando replace
#finance['organismo'] = finance['organismo'].replace(categorias)
finance.loc[:, 'organismo'] = finance['organismo'].replace(categorias)

# Reemplazar cualquier otro número que no esté en el diccionario con '-'
#finance['organismo'] = finance['organismo'].replace(to_replace=r'^\d+$', value='-', regex=True)
finance.loc[:, 'organismo'] = finance['organismo'].replace(to_replace=r'^\d+$', value='-', regex=True)

# Eliminar la fila donde rango_ingresos es '0'
finance = finance[finance['organismo'] != '-']

# Obtener el conteo de registros por cada categoría en la columna 'organismo'
conteo_organismos = finance['organismo'].value_counts().reset_index()
conteo_organismos.columns = ['organismo', 'conteo']


# Asegurarse de que todas las categorías estén presentes en el conteo, incluso si tienen un conteo de 0
for categoria in categorias.values():
    if categoria not in conteo_organismos['organismo'].unique():
        nuevo_registro = pd.DataFrame({'organismo': [categoria], 'conteo': [0]})
        conteo_organismos = pd.concat([conteo_organismos, nuevo_registro], ignore_index=True)

# Ordenar el DataFrame por las categorías en orden ascendente
conteo_organismos = conteo_organismos.sort_values(by='organismo').reset_index(drop=True)

print(conteo_organismos)

    organismo  conteo
0  BANJERCITO       3
1        CNBV     660
2      CONAVI       0
3    FOVISSTE      43
4   INFONAVIT     670
5       INSUS       0


C:\Users\yoe11\AppData\Local\Temp\ipykernel_13872\2664051192.py:3: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['1' '1' '1' ... '3' '4' '4']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.



In [11]:
# Crear un DataFrame con el conteo de registros por cada organismo
df_conteo_organismos = finance['organismo'].value_counts().reset_index()
df_conteo_organismos.columns = ['organismo', 'conteo']

# Calcular el total de registros en el DataFrame original
total_registros = len(finance)

# Calcular el promedio que ocupa cada organismo con respecto al total de registros
df_conteo_organismos['promedio'] = round((df_conteo_organismos['conteo'] * 100) / total_registros,2)
df_conteo_organismos

,organismo,conteo,promedio
0,INFONAVIT,670,48.69
1,CNBV,660,47.97
2,FOVISSTE,43,3.12
3,BANJERCITO,3,0.22


In [12]:
# Calcular el total de registros en el DataFrame original
total_registros = len(finance)

# Calcular el promedio que ocupa cada organismo con respecto al total de registros
conteo_organismos['promedio'] = round((conteo_organismos['conteo'] * 100) / total_registros,2)
conteo_organismos

,organismo,conteo,promedio
0,BANJERCITO,3,0.22
1,CNBV,660,47.97
2,CONAVI,0,0.00
3,FOVISSTE,43,3.12
4,INFONAVIT,670,48.69
5,INSUS,0,0.00


In [13]:
f_2022 = finance.size
cont = df_conteo_organismos['conteo'].sum()
print("Hay ", f_2022, " registros en la base")
print("Hay ", cont, " registros al contar")

Hay  2752  registros en la base
Hay  1376  registros al contar


In [14]:
# Definir el diccionario de orden de organismos
orden_organismos = {
    'INFONAVIT': 1,
    'CNBV': 2,
    'FOVISSTE': 3,
    'BANJERCITO': 4
}

#df_conteo_organismos['orden'] = df_conteo_organismos['organismo'].map(orden_organismos)
df_conteo_organismos.loc[:, 'orden'] = df_conteo_organismos['organismo'].map(orden_organismos)

# Ordenar el DataFrame por el orden de los organismos
df_conteo_organismos = df_conteo_organismos.sort_values(by='orden')
df_conteo_organismos = df_conteo_organismos.drop(columns=['orden'])
df_conteo_organismos

,organismo,conteo,promedio
0,INFONAVIT,670,48.69
1,CNBV,660,47.97
2,FOVISSTE,43,3.12
3,BANJERCITO,3,0.22


In [15]:

labels = df_conteo_organismos['organismo']
values = df_conteo_organismos['promedio']
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']

fig = go.Figure()
fig.add_trace(go.Bar(
        x=labels, 
        y=values, 
        marker_color=colores,
        text=values,  # Texto que se mostrará en las barras (porcentaje)
        textposition='outside',  # Posición del texto (puede ser 'inside' o 'outside')
        texttemplate='%{text:.2f}%', 
    )
)    
fig.update_layout(
    #barmode='stack', 
    #title='Porcentaje de créditos por organismo (2022)',
    #title_font_size = 22,
    yaxis=dict(
        title='%', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y         
    ),  margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    legend=dict(
        yanchor="top",
        y=0.99,
    )
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets\graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    # Configuración personalizada para la gráfica
    config = {
        'displaylogo': False,  # Ocultar el logo de Plotly
        'modeBarButtonsToRemove': [
            'toImage',       # Botón para guardar como imagen
            'select2d',      # Box select
            'lasso2d',       # Lasso select
            'resetScale2d',  # Reset Axes
        ]
    }

    # Guardar la gráfica como archivo HTML
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html', config=config)
guardar_grafico_como_html(fig, 'g_bar_creditos_organismo_2023', carpeta='assets\graficas')
fig.show()

<>:30: SyntaxWarning:

invalid escape sequence '\g'

<>:47: SyntaxWarning:

invalid escape sequence '\g'

<>:30: SyntaxWarning:

invalid escape sequence '\g'

<>:47: SyntaxWarning:

invalid escape sequence '\g'

C:\Users\yoe11\AppData\Local\Temp\ipykernel_13872\3536505722.py:30: SyntaxWarning:

invalid escape sequence '\g'

C:\Users\yoe11\AppData\Local\Temp\ipykernel_13872\3536505722.py:47: SyntaxWarning:

invalid escape sequence '\g'



## 2024

In [16]:
finance = df_financiamientos_2024_vallarta[['id','organismo']]
finance

,id,organismo
0,1,1
1,2,1
2,3,1
3,4,1
4,5,1
...,...,...
1175,1176,2
1176,1177,3
1177,1178,3
1178,1179,3


In [17]:
# Convertir la columna 'organismo' a cadena de texto (str)
#finance['organismo'] = finance['organismo'].astype(str)
finance.loc[:, 'organismo'] = finance['organismo'].astype(str)

# Crear un diccionario con las correspondencias entre número y categoría
categorias = {
    '1': 'INFONAVIT',
    '2': 'CNBV',
    '3': 'FOVISSTE',
    '5': 'CONAVI',
    '7': 'BANJERCITO',
    '26': 'INSUS'
}
# Reemplazar los valores de 'organismo' con el texto correspondiente utilizando replace
#finance['organismo'] = finance['organismo'].replace(categorias)
finance.loc[:, 'organismo'] = finance['organismo'].replace(categorias)

# Reemplazar cualquier otro número que no esté en el diccionario con '-'
#finance['organismo'] = finance['organismo'].replace(to_replace=r'^\d+$', value='-', regex=True)
finance.loc[:, 'organismo'] = finance['organismo'].replace(to_replace=r'^\d+$', value='-', regex=True)

# Eliminar la fila donde rango_ingresos es '0'
finance = finance[finance['organismo'] != '-']

# Obtener el conteo de registros por cada categoría en la columna 'organismo'
conteo_organismos = finance['organismo'].value_counts().reset_index()
conteo_organismos.columns = ['organismo', 'conteo']


# Asegurarse de que todas las categorías estén presentes en el conteo, incluso si tienen un conteo de 0
for categoria in categorias.values():
    if categoria not in conteo_organismos['organismo'].unique():
        nuevo_registro = pd.DataFrame({'organismo': [categoria], 'conteo': [0]})
        conteo_organismos = pd.concat([conteo_organismos, nuevo_registro], ignore_index=True)

# Ordenar el DataFrame por las categorías en orden ascendente
conteo_organismos = conteo_organismos.sort_values(by='organismo').reset_index(drop=True)

print(conteo_organismos)

    organismo  conteo
0  BANJERCITO       1
1        CNBV     507
2      CONAVI       0
3    FOVISSTE      33
4   INFONAVIT     624
5       INSUS       0


C:\Users\yoe11\AppData\Local\Temp\ipykernel_13872\2664051192.py:3: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['1' '1' '1' ... '3' '3' '4']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.



In [18]:
# Crear un DataFrame con el conteo de registros por cada organismo
df_conteo_organismos = finance['organismo'].value_counts().reset_index()
df_conteo_organismos.columns = ['organismo', 'conteo']

# Calcular el total de registros en el DataFrame original
total_registros = len(finance)

# Calcular el promedio que ocupa cada organismo con respecto al total de registros
df_conteo_organismos['promedio'] = round((df_conteo_organismos['conteo'] * 100) / total_registros,2)
df_conteo_organismos

,organismo,conteo,promedio
0,INFONAVIT,624,53.56
1,CNBV,507,43.52
2,FOVISSTE,33,2.83
3,BANJERCITO,1,0.09


In [19]:
# Calcular el total de registros en el DataFrame original
total_registros = len(finance)

# Calcular el promedio que ocupa cada organismo con respecto al total de registros
conteo_organismos['promedio'] = round((conteo_organismos['conteo'] * 100) / total_registros,2)
conteo_organismos

,organismo,conteo,promedio
0,BANJERCITO,1,0.09
1,CNBV,507,43.52
2,CONAVI,0,0.00
3,FOVISSTE,33,2.83
4,INFONAVIT,624,53.56
5,INSUS,0,0.00


In [20]:
f_2022 = finance.size
cont = df_conteo_organismos['conteo'].sum()
print("Hay ", f_2022, " registros en la base")
print("Hay ", cont, " registros al contar")

Hay  2330  registros en la base
Hay  1165  registros al contar


In [21]:
# Definir el diccionario de orden de organismos
orden_organismos = {
    'INFONAVIT': 1,
    'CNBV': 2,
    'FOVISSTE': 3,
    'BANJERCITO': 4
}

#df_conteo_organismos['orden'] = df_conteo_organismos['organismo'].map(orden_organismos)
df_conteo_organismos.loc[:, 'orden'] = df_conteo_organismos['organismo'].map(orden_organismos)

# Ordenar el DataFrame por el orden de los organismos
df_conteo_organismos = df_conteo_organismos.sort_values(by='orden')
df_conteo_organismos = df_conteo_organismos.drop(columns=['orden'])
df_conteo_organismos

,organismo,conteo,promedio
0,INFONAVIT,624,53.56
1,CNBV,507,43.52
2,FOVISSTE,33,2.83
3,BANJERCITO,1,0.09


In [22]:

labels = df_conteo_organismos['organismo']
values = df_conteo_organismos['promedio']
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']

fig = go.Figure()
fig.add_trace(go.Bar(
        x=labels, 
        y=values, 
        marker_color=colores,
        text=values,  # Texto que se mostrará en las barras (porcentaje)
        textposition='outside',  # Posición del texto (puede ser 'inside' o 'outside')
        texttemplate='%{text:.2f}%', 
    )
)    
fig.update_layout(
    #barmode='stack', 
    #title='Porcentaje de créditos por organismo (2022)',
    #title_font_size = 22,
    yaxis=dict(
        title='%', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y         
    ),  margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    legend=dict(
        yanchor="top",
        y=0.99,
    )
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets\graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    # Configuración personalizada para la gráfica
    config = {
        'displaylogo': False,  # Ocultar el logo de Plotly
        'modeBarButtonsToRemove': [
            'toImage',       # Botón para guardar como imagen
            'select2d',      # Box select
            'lasso2d',       # Lasso select
            'resetScale2d',  # Reset Axes
        ]
    }

    # Guardar la gráfica como archivo HTML
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html', config=config)
guardar_grafico_como_html(fig, 'g_bar_creditos_organismo_2024', carpeta='assets\graficas')
fig.show()

<>:30: SyntaxWarning:

invalid escape sequence '\g'

<>:47: SyntaxWarning:

invalid escape sequence '\g'

<>:30: SyntaxWarning:

invalid escape sequence '\g'

<>:47: SyntaxWarning:

invalid escape sequence '\g'

C:\Users\yoe11\AppData\Local\Temp\ipykernel_13872\1545832123.py:30: SyntaxWarning:

invalid escape sequence '\g'

C:\Users\yoe11\AppData\Local\Temp\ipykernel_13872\1545832123.py:47: SyntaxWarning:

invalid escape sequence '\g'

